In [1]:
import glob
import os


from wordcloud import WordCloud, STOPWORDS
from nltk.corpus import stopwords

import pandas as pd

from PIL import Image 
import matplotlib.pylab as plt

In [2]:
import os
import re
import time
from pathlib import Path
import shutil
import glob
import string
import unicodedata
from datetime import datetime, timedelta

import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from sqlalchemy import create_engine
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from unicodedata import normalize, category

In [3]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx

In [4]:
class ApplyTextRank(object):
    
    def __init__(self, hoy_format=None,  metodo=1):
        #-------------------------- Habilitacion de ruta --------------------------#
        self.folder_news = 'news'
        if not os.path.exists(self.folder_news):
            os.makedirs(self.folder_news)
        
        self.hoy = datetime.today()
        self.hoy_format = self.hoy.strftime('%d%m%Y') if hoy_format is None else hoy_format
        self.newsletter_path = os.path.join(self.folder_news, self.hoy_format, 'consolidado')
        
        path_ = os.path.join(self.newsletter_path, 'consolidado.csv'.format(metodo))
        data = pd.read_csv(path_, encoding='UTF-8')
        data = data[data['tipo'] == 'original']
        #display(data)
        
        self._sentences_origen = [oracion for oracion in data['oracion']]
        self._sentences = []
        self._stopwords = stopwords.words('spanish')

        
    def remove_stopwords(self):
        self._sentences = [
            [words for words in sentence.split() if words not in self._stopwords] 
            for sentence in self._sentences_origen
        ]

In [5]:
apply_text_rank = ApplyTextRank()

In [6]:
#apply_text_rank._sentences_origen

In [7]:
apply_text_rank.remove_stopwords()

In [8]:
%%time

w2v = Word2Vec(
    apply_text_rank._sentences, 
    size=1,
    min_count=1,
    iter=1000
)

Wall time: 4.27 s


In [9]:
sentence_embeddings = [
    [w2v[word][0] for word in words] 
    for words in apply_text_rank._sentences
]

<ipython-input-9-516a21e910ff>:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  [w2v[word][0] for word in words]


In [10]:
max_len =max(
    [len(tokens) for tokens in apply_text_rank._sentences]
)
max_len

45

In [11]:
sentence_embeddings = [
    np.pad(embedding,(0,max_len-len(embedding)),'constant') 
    for embedding in sentence_embeddings
]

In [12]:
similarity_matrix = np.zeros([len(apply_text_rank._sentences), len(apply_text_rank._sentences)])
similarity_matrix.shape

(240, 240)

In [13]:
for i, row_embedding in enumerate(sentence_embeddings):
    for j, column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j] = 1 - spatial.distance.cosine(row_embedding, column_embedding)

### PAGE RANK

In [14]:
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph)

In [15]:
_scores = pd.Series(scores).reset_index()
_scores.columns = ['orden', 'score']
_scores

,orden,score
0,0,0.004622
1,1,0.004379
2,2,0.004554
3,3,0.004369
4,4,0.004452
...,...,...
235,235,0.003151
236,236,0.004262
237,237,0.003158
238,238,0.004514


### TOP

In [16]:
oraciones = pd.Series(apply_text_rank._sentences_origen).reset_index()
oraciones.columns = ['orden', 'oracion']
oraciones

,orden,oracion
0,0,la pandemia de coronavirus ha dejado a muchas ...
1,1,ante ello el anterior gobierno de francisco s...
2,2,aqui te alcanzamos el link oficial de consult...
3,3,como se sabe el pago del bono 600 empezo en fe...
4,4,hace unas semanas la anterior gestion anuncio...
...,...,...
235,235,por eso estamos incentivando la vacunacion
236,236,seguimos insistiendo en la poblacion se cuide...
237,237,no hay que confiarse insto
238,238,consultado respecto a que regiones presentan c...


In [17]:
_scores = _scores.merge(oraciones, how='left', on='orden')
_scores['oracion'] == _scores['oracion'].apply(lambda _: str(_).strip())
_scores

,orden,score,oracion
0,0,0.004622,la pandemia de coronavirus ha dejado a muchas ...
1,1,0.004379,ante ello el anterior gobierno de francisco s...
2,2,0.004554,aqui te alcanzamos el link oficial de consult...
3,3,0.004369,como se sabe el pago del bono 600 empezo en fe...
4,4,0.004452,hace unas semanas la anterior gestion anuncio...
...,...,...,...
235,235,0.003151,por eso estamos incentivando la vacunacion
236,236,0.004262,seguimos insistiendo en la poblacion se cuide...
237,237,0.003158,no hay que confiarse insto
238,238,0.004514,consultado respecto a que regiones presentan c...


In [18]:
del oraciones

In [19]:
_scores.sort_values(by=['score'], ascending=False).head()

,orden,score,oracion
147,147,0.004669,faltaban solo dos dias para las vacaciones sin...
77,77,0.004666,uno tiene que poner su firma y dar la cara po...
137,137,0.004665,como soma de 24 anos que la semana pasada part...
126,126,0.004659,cancelen las olimpiadas gritaban los manifesta...
12,12,0.004643,ademas te ayudaran a completar los pasos para...


In [20]:
top = _scores.sort_values(by=['score'], ascending=False).head(10)['oracion']
top

147    faltaban solo dos dias para las vacaciones sin...
77      uno tiene que poner su firma y dar la cara po...
137    como soma de 24 anos que la semana pasada part...
126    cancelen las olimpiadas gritaban los manifesta...
12      ademas te ayudaran a completar los pasos para...
87     fiorella molinelli junto a otras 13 personas s...
36      en diferentes locales de vacunacion se han im...
0      la pandemia de coronavirus ha dejado a muchas ...
191    este sabado 31 de julio el minsa informo que s...
16     si tras revisar la web el bono 600 soles verif...
Name: oracion, dtype: object

In [25]:
top = list(set(top))
top[:5]

[' uno tiene que poner su firma y dar la cara porque hay que tomar decisiones sin tener miedo porque la gente se esta muriendo',
 'faltaban solo dos dias para las vacaciones sin embargo los planes se frustraron a raiz de un incidente negativo',
 'este sabado 31 de julio el minsa informo que se superaron los 5 millones de peruanos que han completado su vacunacion contra el covid19 con las dos dosis',
 'cancelen las olimpiadas gritaban los manifestantes subidos en tarimas o furgonetas y rodeados de policias impasibles',
 'la pandemia de coronavirus ha dejado a muchas personas sin empleo y afecto economicamente a miles de hogares']

In [22]:
#_path_consolidado = os.path.join(
#    apply_text_rank.newsletter_path.replace('parseado', 'consolidado'), 'consolidado.csv'
#)

In [23]:
#consolidado = pd.read_csv(_path_consolidado, encoding="utf-8")
#consolidado = consolidado[consolidado['tipo'] == 'original']
#consolidado.head(20)

In [24]:
#consolidado = consolidado.merge(
#    _scores, 
#    how='left', 
#    on='parseado_1'
#)

#consolidado.head()